In [1]:
from Bio import Medline
from collections import defaultdict
abstract = []
source_list = []
with open("pubmed_result_medline.txt") as handle:
    for record in Medline.parse(handle):
#         source = record['SO']
#         source_list.append(source)
        for k, v in record.iteritems():
            if k == 'AB':
                abs_single = record[k]
                abstract.append(abs_single)
#             elif k == "SO":
#                 source = record[k]
#                 source_list.append(source)

abstract_dict = defaultdict(None, (enumerate(abstract)))
            


In [2]:
# This snippet pairs PMID with abstracts
rec_so = {}
with open("pubmed_result_plos_med.txt") as handle:
    for record in Medline.parse(handle):
        if 'AB' in record.keys(): 
            pmid, abstract = record['PMID'], record['AB']
            rec_so[pmid] = abstract
        
        

In [4]:
import pandas as pd
df2 = pd.DataFrame.from_dict(rec_so, orient='index').reset_index()
        

In [5]:
df2.columns = ['PMID', 'Abstract']

In [6]:
df2.head()

,PMID,Abstract
0,16584289,BACKGROUND: Streptococcus suis serotype 2 (S. ...
1,27575534,BACKGROUND: The prevention of falls among olde...
2,27116432,"In this month's editorial, the PLOS Medicine E..."
3,27923049,"BACKGROUND: Each year, over 16,000 patients di..."
4,26348035,BACKGROUND: Home-based HIV testing and counsel...


In [13]:
df2['tokenized'] = [tokenize_abstract(abstract) for abstract in df2['Abstract']]

NameError: global name 'stopwords' is not defined

In [22]:
df2.head()

,PMID,Abstract
0,16584289,BACKGROUND: Streptococcus suis serotype 2 (S. ...
1,27575534,BACKGROUND: The prevention of falls among olde...
2,27116432,"In this month's editorial, the PLOS Medicine E..."
3,27923049,"BACKGROUND: Each year, over 16,000 patients di..."
4,26348035,BACKGROUND: Home-based HIV testing and counsel...


## Need to find a way to pair PMID or article source ID with abstracts

In [15]:
len(abstract_dict)

6387

In [16]:
import pandas as pd
import string
import numpy as np


In [17]:
import nltk

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

##  Do I need to do this?

In [19]:
import os
import glob
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# Lowercase, tokenize and lemmatize all the documents
def tokenize_abstract(abstract):
    clean_abstract = abstract.lower().translate(None, string.punctuation)
    tokens = word_tokenize(clean_abstract)
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tokens]
    clean_words = [w for w in stemmed_words if w not in stopwords.words('english')]
    return ' '.join(clean_words)

In [21]:
tokenized_dict = {}
for k, v in abstract_dict.iteritems():
    token = tokenize_abstract(v)
    tokenized_dict[k] = token

In [23]:
tokenized_dict[0]

u'physiolog function epiderm cell larg determin divers morpholog flower plant special conicalshap petal epiderm cell thought influenc light captur reflect provid pollin grip molecular mechan control conic cell shape remain larg unknown develop liveconfoc imag approach quantifi geometr paramet conic cell arabidopsi thaliana athaliana genet screen identifi katanin ktn1 mutant show phenotyp decreas tip sharpen conic cell furthermor demonstr spike1 rho plant rop gtpase requir final shape format conic cell ktn1 doe livecel imag show wildtyp cell exhibit random orient cortic microtubul array earli development stage display wellord circumferenti orient microtubul array later stage contrast loss ktn1 prevent random microtubul network shift wellord array show filament actin cap typic featur sever plant epiderm cell type includ root hair leaf trichom observ growth apex conic cell dure cell develop moreov genet pharmacolog data suggest microtubul actin requir conic cell shape togeth result provid

In [24]:
df = pd.DataFrame(np.array([abstract_dict.values()]).T)

In [25]:
df.columns = ['abstract']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
df['tokenized'] = tokenized_dict.values()

In [45]:
df.head()

,abstract,tokenized,vectorized
0,The physiological functions of epidermal cells...,physiolog function epiderm cell larg determin ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
1,Vertical growth of plants is a dynamic process...,vertic growth plant dynam process influenc gen...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
2,Preventive chemotherapy has long been practice...,prevent chemotherapi long practic nematod para...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
3,Traditional genome-wide scans for positive sel...,tradit genomewid scan posit select main uncov ...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."
4,Ribosomal DNA is one of the most variable regi...,ribosom dna one variabl region human genom res...,"(0, 11435)\t0.0525712094045\n (0, 6169)\t0...."


In [43]:
tfid = TfidfVectorizer(stop_words=stop, max_features=20000, min_df=2)
vectorized_abstracts = tfid.fit_transform(df['tokenized'])

In [44]:
df['vectorized'] = vectorized_abstracts

In [46]:
features = tfid.get_feature_names()

[u'000',
 u'000003',
 u'00001',
 u'00002',
 u'00003',
 u'00004',
 u'00005',
 u'00006',
 u'00007',
 u'0001',
 u'00015',
 u'00019',
 u'0002',
 u'00023',
 u'00025',
 u'0003',
 u'00031',
 u'0004',
 u'0005',
 u'0006',
 u'0007',
 u'0008',
 u'0009',
 u'001',
 u'0010',
 u'0011',
 u'0012',
 u'0013',
 u'0014',
 u'0015',
 u'0016',
 u'002',
 u'0021',
 u'0023',
 u'0025',
 u'003',
 u'0032',
 u'0034',
 u'0035',
 u'0039',
 u'004',
 u'0041',
 u'0043',
 u'0045',
 u'0047',
 u'005',
 u'006',
 u'007',
 u'008',
 u'009',
 u'01',
 u'010',
 u'011',
 u'012',
 u'013',
 u'015',
 u'016',
 u'017',
 u'018',
 u'019',
 u'02',
 u'020',
 u'0208',
 u'022',
 u'023',
 u'024',
 u'025',
 u'026',
 u'027',
 u'03',
 u'030',
 u'031',
 u'032',
 u'034',
 u'035',
 u'036',
 u'037',
 u'038',
 u'039',
 u'04',
 u'041',
 u'042',
 u'043',
 u'044',
 u'046',
 u'047',
 u'048',
 u'049',
 u'05',
 u'050',
 u'051',
 u'052',
 u'053',
 u'054',
 u'055',
 u'056',
 u'057',
 u'058',
 u'059',
 u'06']

In [ ]:
cosine_similarity(vectorized_abstracts[0], vectorized_abstracts[1])

In [28]:
from sklearn.decomposition import NMF


In [ ]:

import sys

from Bio import Entrez

# *Always* tell NCBI who you are
Entrez.email = "vpineda@uw.edu"

def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print "An error occurred while retrieving the annotations."
        print "The error returned was %s" % e
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    print "Retrieved %d annotations for %d genes" % (len(annotations),
            len(id_list))

    return annotations

In [ ]:
def print_data(annotation):
    for gene_data in annotation:
        gene_id = gene_data["Id"]
        gene_symbol = gene_data["NomenclatureSymbol"]
        gene_name = gene_data["Description"]
        print "ID: %s - Gene Symbol: %s - Gene Name: %s" % (gene_id, gene_symbol, gene_name)

In [29]:
df_genes = pd.read_csv("hgnc_complete_set.txt", delimiter="\t")

/Applications/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (32,34,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
df_genes.head()

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,merops,imgt,iuphar,kznf_gene_catalog,mamit-trnadb,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,...,I43.950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,19q13.43,19q13.43,FLJ23569,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,URS00007E4F6E
2,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,10q11.23,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,...,I39.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGNC:27057,A2M-AS1,A2M antisense RNA 1 (head to head),non-coding RNA,"RNA, long non-coding",Approved,12p13.31,12p13.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,URS00001F234A


In [31]:
df_genes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42238 entries, 0 to 42237
Data columns (total 49 columns):
hgnc_id                     42238 non-null object
symbol                      42238 non-null object
name                        42238 non-null object
locus_group                 42238 non-null object
locus_type                  42238 non-null object
status                      42238 non-null object
location                    41074 non-null object
location_sortable           41074 non-null object
alias_symbol                20794 non-null object
alias_name                  6113 non-null object
prev_symbol                 11365 non-null object
prev_name                   20703 non-null object
gene_family                 18883 non-null object
gene_family_id              18883 non-null object
date_approved_reserved      41077 non-null object
date_symbol_changed         8921 non-null object
date_name_changed           22166 non-null object
date_modified               42230 non-null 

In [36]:
df_ucsc = pd.read_csv('ucsc_downloads/gene.txt', sep='\t', header=None)

In [37]:
df_ucsc.head()

,0,1,2,3,4,5,6
0,1,Nkx2-6,NaN,NaN,XM_127732,NaN,10090
1,2,Nkx2-2,18088.0,NM_010919,U31566,NaN,10090
2,3,Nkx2-9,18094.0,NM_008701,NM_008701,NaN,10090
3,4,Hmx1,15371.0,NM_010445,NM_010445,NaN,10090
4,5,Foxo1,56458.0,NM_019739,NM_019739,NaN,10090


In [39]:
df_ucsc.columns = ['n', 'gene_name', 'locus_link', 'ref_seq_num', 'genbank', 'uniprot', 'taxon']

In [40]:
df_ucsc.head()

,n,gene_name,locus_link,ref_seq_num,genbank,uniprot,taxon
0,1,Nkx2-6,NaN,NaN,XM_127732,NaN,10090
1,2,Nkx2-2,18088.0,NM_010919,U31566,NaN,10090
2,3,Nkx2-9,18094.0,NM_008701,NM_008701,NaN,10090
3,4,Hmx1,15371.0,NM_010445,NM_010445,NaN,10090
4,5,Foxo1,56458.0,NM_019739,NM_019739,NaN,10090


In [41]:
len(df_ucsc['gene_name'].values)

20413

In [58]:
gene_ucsc = set([name.lower() for name in df_ucsc['gene_name']])

AttributeError: 'float' object has no attribute 'lower'

In [51]:
gene_pairs = []
for abstract in df['tokenized']:
    pairs = []
    for word in abstract:
        if word in gene_ucsc:
            pairs.append(word)
    if len(pairs) > 2:
        gene_pairs.append(pairs)
    else:
        gene_pairs.append(None)


In [53]:
gene_ucsc

{nan,
 'Prdx6-rs1',
 'Plekhg1',
 'Hspbap1',
 'AW208599',
 'Plekhg4',
 'Hnrpab',
 '2600003E23Rik',
 '2810443J12Rik',
 'Tcfe2a',
 'Mash2l',
 'LOC433132',
 'Bckdk',
 '2700067E09Rik',
 'Ube2d1',
 'Shank3',
 'LOC235300',
 'LOC330189',
 '2010011I20Rik',
 'Syt8',
 'Syt9',
 'Mm.149980',
 'Cyln2',
 'Mm.32613',
 'Mug1',
 'Syt1',
 'Syt2',
 'Syt3',
 'Syt4',
 'Syt5',
 'Syt6',
 '9030414G15Rik',
 '4933407D04Rik',
 'Prr3',
 'BC024322',
 'Sult1c2',
 'Tom1',
 'Gstk1',
 'Aqp11',
 'Aqp12',
 '1810010H24Rik',
 'NUP98',
 'LOC230983',
 'SP2',
 'Slc25a18',
 'Abcb1a',
 'Pcdha4',
 'Klc3',
 'D830013H12',
 'Sult1c1',
 'Dag1',
 'Hnrpa3',
 'Slc25a16',
 '2010005O13Rik',
 'mCG22352.3',
 'RAB40C',
 'RAB40B',
 'Sugt1',
 'Exosc10',
 'Cebpa',
 'Cebpg',
 'Cebpe',
 'Cebpd',
 'A830006F12Rik',
 'Slc25a12',
 '6720401G13Rik',
 'D4Ertd432e',
 'LOC434178',
 '4930543L23Rik',
 'Car3',
 'Cebpz',
 'Polr2j',
 'Slc27a6',
 'TC1568695',
 'F830022J16Rik',
 'Arl1',
 'Zfp712',
 'Car1',
 'Pcsk4',
 '2810037C03Rik',
 'ATP2A3',
 'Mllt4',
 'Car6